In [1]:
%pip install google-play-scraper


     ---------------------------------------- 50.2/50.2 kB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

In [3]:
%pip install langchain langchain-community langchain-openai


  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
     ---------------------------------------- 90.6/90.6 kB 2.6 MB/s eta 0:00:00
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.12.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.3-py3-none-any.whl.metadata (9.7 kB)
     ---------------------------------------- 60.8/60.8 kB 1.6 MB/s eta 0:00:00
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
     ---------------------------------------- 77.6/77.6 kB 2.2 MB/s eta 0:00:00
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.10.1 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.2.6 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"


In [9]:
from google_play_scraper import reviews, Sort

APP_ID = "com.application.zomato"

result, _ = reviews(
    APP_ID,
    lang='en',
    country='in',
    sort=Sort.NEWEST,
    count=50
)

for r in result:
    print("Date:", r["at"])
    print("Rating:", r["score"])
    print("Review:", r["content"])
    print("-" * 10)



Date: 2026-01-22 00:19:39
Rating: 5
Review: on time delivery, thank you Zomato
----------
Date: 2026-01-22 00:15:50
Rating: 4
Review: excellent
----------
Date: 2026-01-22 00:10:59
Rating: 1
Review: bakwash hai no refund no product test
----------
Date: 2026-01-22 00:10:46
Rating: 5
Review: good experience
----------
Date: 2026-01-22 00:03:55
Rating: 5
Review: good service neet
----------
Date: 2026-01-21 23:56:56
Rating: 5
Review: nice
----------
Date: 2026-01-21 23:50:11
Rating: 5
Review: amazing experience
----------
Date: 2026-01-21 23:47:20
Rating: 5
Review: good
----------
Date: 2026-01-21 23:45:36
Rating: 5
Review: always help me to fulfil my craving.
----------
Date: 2026-01-21 23:38:23
Rating: 5
Review: wooooo
----------
Date: 2026-01-21 23:34:55
Rating: 5
Review: very nice
----------
Date: 2026-01-21 23:34:47
Rating: 5
Review: ok
----------
Date: 2026-01-21 23:32:39
Rating: 5
Review: give me best offer for me
----------
Date: 2026-01-21 23:27:22
Rating: 1
Review: it's a fraud

In [14]:
df=pd.DataFrame(result)
df['date'] = df['at'].dt.date
df[['date', 'score', 'content']].head()


NameError: name 'pd' is not defined

In [43]:
base_path = "data/zomato"
os.makedirs(base_path, exist_ok=True)


In [44]:
for date, daily_df in df.groupby('date'):
    file_path = f"{base_path}/{date}.csv"
    daily_df[['date', 'score', 'content']].to_csv(file_path, index=False)


In [49]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=128
)




In [50]:
df = pd.read_csv("/content/data/zomato/2026-01-08.csv")

sample_review = df.iloc[0]["content"]

print(sample_review)


excellent


In [51]:
prompt = PromptTemplate(template="""
You are analyzing Google Play Store app reviews.

Your task is to extract ALL concrete topics related to:
- specific issues
- specific complaints
- specific requests
- specific suggestions
- specific feedback about features, service, or experience

CRITICAL RULES:
- Topics MUST refer to a concrete aspect (delivery, food, app, support, pricing, etc.)
- Do NOT return generic phrases like:
  "Positive feedback", "Negative feedback", "Good experience", "Bad experience"
- Do NOT summarize sentiment
- Topics must be short, specific phrases (3–6 words)
- Always extract at least one concrete topic
- Multiple topics are allowed

Review:
{review}

Return topics as a bullet list.
""")



In [52]:
chain=prompt | model | StrOutputParser()

response = chain.invoke({
    "review": sample_review
})

print(response)


- No specific topics identified


In [53]:
canonical_topics = set([
    "Delivery issue",
    "Delivery partner rude",
    "Late delivery",
    "Food quality issue",
    "App performance issue",
    "App user interface"
])


In [54]:
canonical_prompt = PromptTemplate(template="""
You are a topic canonicalization agent for app review analysis.

Your task:
Given a newly extracted topic and a list of existing canonical topics,
decide the BEST canonical topic.

Rules:
- Merge aggressively
- If the new topic is even loosely similar to an existing topic, MERGE
- Prefer existing topics over creating new ones
- Canonical topics must be short, neutral, and stable over time
- Avoid synonyms, paraphrases, or stylistic variations

If merging:
- Return the EXACT existing topic name

If creating a new topic:
- Rewrite it into a clean, neutral canonical form

Inputs:
New topic:
{new_topic}

Existing canonical topics:
{existing_topics}

Return ONLY ONE topic name. No explanation.
""")


In [55]:
canonical_chain = canonical_prompt | model | StrOutputParser()

testing with real eg


In [56]:
existing = list(canonical_topics)

response = canonical_chain.invoke({
    "new_topic": "Smooth performance",
    "existing_topics": existing
})

print(response)


App performance issue


In [57]:
response = canonical_chain.invoke({
    "new_topic": "UI design",
    "existing_topics": existing
})

print(response)


App user interface


In [58]:
response = canonical_chain.invoke({
    "new_topic": "Refund not received",
    "existing_topics": existing
})

print(response)


Refund not received


In [59]:
canonical_topics.add(response)


In [60]:
canonical_topics

{'App performance issue',
 'App user interface',
 'Delivery issue',
 'Delivery partner rude',
 'Food quality issue',
 'Late delivery',
 'Refund not received'}

In [61]:
topic_chain = chain


In [62]:
import os

data_dir = "/content/data/zomato"
daily_topic_store = {}

for file in sorted(os.listdir(data_dir)):
    if file.endswith(".csv"):
        date = file.replace(".csv", "")
        print("Processing:", date)

        daily_topic_store[date] = process_daily_reviews(
            csv_path=f"{data_dir}/{file}",
            topic_chain=topic_chain,
            canonical_chain=canonical_chain,
            canonical_topics=canonical_topics
        )


Processing: 2026-01-08


Processing reviews: 100%|██████████| 35/35 [01:40<00:00,  2.88s/it]


Processing: 2026-01-09


Processing reviews: 100%|██████████| 50/50 [01:46<00:00,  2.13s/it]


In [28]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [65]:
trend_df = pd.DataFrame(daily_topic_store).fillna(0).astype(int)
trend_df


,2026-01-08,2026-01-09
App performance issue,5,4
No specific suggestions mentioned,82,131
App user interface,11,8
Delivery issue,7,7
Late delivery,3,5
Food quality issue,7,2
Refund not received,1,0
Delivery partner rude,0,1


In [67]:

os.makedirs("output", exist_ok=True)

In [69]:
output_path = "output/zomato_topic_trend_sample.csv"
trend_df.to_csv(output_path)


